## 1.先转Vector ； 2.再转Array

In [20]:
var memory = 32
var overhead = (memory*1024*0.07).toInt
var ememory = (memory * 1024 - overhead)/1024 + 1
val spark = SparkSession
      .builder      //创建spark会话
      .master("yarn")  //设置本地模式
      .appName("kaggle_data_convertion_scala")  //设置名称
      .config("spark.executor.instances","10")
      .config("spark.dynamicAllocation.maxExecutors", "20")
      .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
      .config("spark.kryoserializer.buffer.max", "512m")
      .config("spark.executor.memory","32g")
      .config("spark.driver.memory","32g")
      .config("spark.locality.wait","15")
      .config("spark.yarn.executor.memoryOverhead", String.valueOf(overhead))
      .getOrCreate() //创建会话变量

memory = 32
overhead = 2293
ememory = 30
spark = org.apache.spark.sql.SparkSession@41af84d4


Spark application_1631593462186_0510: Some(http://Medusa002:4045)

In [21]:
%AddJar  file:///home/luokaijing/spark-bsp_2.11-2.4.0.jar

Using cached version of spark-bsp_2.11-2.4.0.jar


In [22]:
val path = "/user/luokaijing/classification_BSP/raw_kaggle/train.csv"

path = /user/luokaijing/classification_BSP/raw_kaggle/train.csv


/user/luokaijing/classification_BSP/raw_kaggle/train.csv

In [23]:
var rawDF = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(path)

rawDF = [lgbmcpu: double, lgbmgpu: double ... 26 more fields]


[lgbmcpu: double, lgbmgpu: double ... 26 more fields]

In [24]:
val cols1 = Array("lgbmcpu", "lgbmgpu", "xgb", "catgpu", "catcpu", "histcpu", "lgbmseed", "lgbmseed2", "lgbmseed3", "lgbmseed4", "voting_5cat", "voting_7cat", "voting_5xgb", "catseed", "lgbmgpu_seed", "lgbm1_seed", "lgbm2_seed", "lgbm3_seed", "xgb_seed", "hist_seed", "xgb_op", "lgbm_k", "lgbm2_k", "cat_k", "xgb_k", "hist_k", "xgb_top")
val cols = Array("lgbmcpu", "lgbmgpu", "xgb", "catgpu", "catcpu", "histcpu", "lgbmseed", "lgbmseed2", "lgbmseed3", "lgbmseed4", "voting_5cat", "voting_7cat", "voting_5xgb", "catseed", "lgbmgpu_seed", "lgbm1_seed", "lgbm2_seed", "lgbm3_seed", "xgb_seed", "hist_seed", "xgb_op", "lgbm_k", "lgbm2_k", "cat_k", "xgb_k", "hist_k", "xgb_top")

cols1 = Array(lgbmcpu, lgbmgpu, xgb, catgpu, catcpu, histcpu, lgbmseed, lgbmseed2, lgbmseed3, lgbmseed4, voting_5cat, voting_7cat, voting_5xgb, catseed, lgbmgpu_seed, lgbm1_seed, lgbm2_seed, lgbm3_seed, xgb_seed, hist_seed, xgb_op, lgbm_k, lgbm2_k, cat_k, xgb_k, hist_k, xgb_top)
cols = Array(lgbmcpu, lgbmgpu, xgb, catgpu, catcpu, histcpu, lgbmseed, lgbmseed2, lgbmseed3, lgbmseed4, voting_5cat, voting_7cat, voting_5xgb, catseed, lgbmgpu_seed, lgbm1_seed, lgbm2_seed, lgbm3_seed, xgb_seed, hist_seed, xgb_op, lgbm_k, lgbm2_k, cat_k, xgb_k, hist_k, xgb_top)


[lgbmcpu, lgbmgpu, xgb, catgpu, catcpu, histcpu, lgbmseed, lgbmseed2, lgbmseed3, lgbmseed4, voting_5cat, voting_7cat, voting_5xgb, catseed, lgbmgpu_seed, lgbm1_seed, lgbm2_seed, lgbm3_seed, xgb_seed, hist_seed, xgb_op, lgbm_k, lgbm2_k, cat_k, xgb_k, hist_k, xgb_top]

In [25]:
import org.apache.spark.ml.feature.{LabeledPoint, VectorAssembler}
import org.apache.spark.ml.linalg.{Vector, Vectors}
import org.apache.spark.ml.stat.Summarizer
import org.apache.spark.sql.{Row, SparkSession}

In [26]:
//合并成 vector
val assembler = new VectorAssembler()
  .setInputCols(cols)
  .setOutputCol("features")
val dataset = assembler.transform(rawDF)

assembler = vecAssembler_76b2aab64252
dataset = [lgbmcpu: double, lgbmgpu: double ... 27 more fields]


[lgbmcpu: double, lgbmgpu: double ... 27 more fields]

In [27]:
val dataset1 = dataset.select($"features",$"target").withColumnRenamed("target","label")

dataset1 = [features: vector, label: int]


[features: vector, label: int]

In [28]:
import org.apache.spark.sql.types._

In [30]:
val x = StructType(Seq(
  StructField("features", ArrayType(DoubleType,true),true),
  StructField("label", IntegerType,true)
))

x = StructType(StructField(features,ArrayType(DoubleType,true),true), StructField(label,IntegerType,true))


StructType(StructField(features,ArrayType(DoubleType,true),true), StructField(label,IntegerType,true))

In [33]:
val BspDF_RDD1 = dataset1.rdd.map(row => Row(row(0).asInstanceOf[Vector].toArray, row(1)))

BspDF_RDD1 = MapPartitionsRDD[15] at map at <console>:63


MapPartitionsRDD[15] at map at <console>:63

In [34]:
val BspDF_input = spark.createDataFrame(BspDF_RDD1, x)

BspDF_input = [features: array<double>, label: int]


[features: array<double>, label: int]

In [35]:
BspDF_input.write.parquet("./classification_BSP/train.parquet")

In [19]:
spark.stop()